<strong><h3 style="font-family:Comic Sans MS;color: #FF5733">Web Scraping</h3></strong>

<strong><span style="color: #FF33F0">1. Install dependencies (pandas, requests...)</span></strong>

In [8]:
import pandas as pd 
import requests

<strong><span style="color: #FF33F0">2. Download HTML. We are going to scrape the website https://vaz.io/tesla that contains Tesla annual/quarterly revenue history and growth rate from 2010 to 2023.</span></strong>

In [2]:
# specify the url
url = 'https://vaz.io/tesla/'

# send a GET request to the server and store the server's response
response = requests.get(url)
"""
import time
# If no information is extracted, the solution suggests to connect as anonymous, as follows:
if "403 Forbidden" in response:
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"}
    request = requests.get(url, headers = headers)
    time.sleep(10)
    response = request.text
"""
# display the server's response, should be 200 OK
response

<Response [200]>

<strong><span style="color: #FF33F0">3. Transform the HTML: the next step to start extracting the information is to transform it into a structured object.</span></strong>

In [3]:
# import the BeautifulSoup library to parse HTML content
from bs4 import BeautifulSoup

# parse the html content of the web page
soup = BeautifulSoup(response.content, 'html.parser') # function takes 2 arguments: HTML content to be parsed and the parser library to use 'html.parser' which is a built-in python library for parsing HTML and XML
soup.prettify() # the prettify method converts the parsed HTML into a nicely formatted string with one tag per line and proper indentation for nested tags, making it easier to read and understand

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html class="TridactylThemeDefault js flexbox canvas canvastext webgl no-touch geolocation postmessage no-websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients no-cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" style="">\n <script id="custom-useragent-string-page-script" type="text/javascript">\n </script>\n <!--<![endif]-->\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>\n  <meta charset="utf-8"/>\n  <meta content="IE=ed

<strong><span style="color: #FF33F0">3.1. Find all the tables.</span></strong>

In [4]:
tables = soup.find_all('table')
tables

[<table class="historical_data_table table">
 <thead>
 <tr>
 <th colspan="2" style="text-align:center">Tesla Annual Revenue<br/><span style="font-size:14px;">(Millions of US $)</span></th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td style="text-align:center">2023</td>
 <td style="text-align:center">$96,773</td>
 </tr>
 <tr>
 <td style="text-align:center">2022</td>
 <td style="text-align:center">$81,462</td>
 </tr>
 <tr>
 <td style="text-align:center">2021</td>
 <td style="text-align:center">$53,823</td>
 </tr>
 <tr>
 <td style="text-align:center">2020</td>
 <td style="text-align:center">$31,536</td>
 </tr>
 <tr>
 <td style="text-align:center">2019</td>
 <td style="text-align:center">$24,578</td>
 </tr>
 <tr>
 <td style="text-align:center">2018</td>
 <td style="text-align:center">$21,461</td>
 </tr>
 <tr>
 <td style="text-align:center">2017</td>
 <td style="text-align:center">$11,759</td>
 </tr>
 <tr>
 <td style="text-align:center">2016</td>
 <td style="text-align:center">$7,000</td>
 </tr>
 <

<strong><span style="color: #FF33F0">3.2. Find the table with the quarterly evolution.</span></strong>

In [5]:
for index, table in enumerate(tables):
    if 'Tesla Quarterly Revenue' in str(table):
        table_index = index
        break

<strong><span style="color: #FF33F0">3.3. Store the data in a DataFrame.</span></strong>

In [6]:
# create a dataframe
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
for row in tables[table_index].tbody.find_all('tr'):
    col = row.find_all('td')
    if (col != []):
        Date = col[0].text
        Revenue = col[1].text.replace('$', '').replace(',', '')
        tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({'Date': [Date], 'Revenue': [Revenue]})], ignore_index=True)

# display the dataframe
print(tesla_revenue.head(5)) # display the first 5 rows of the dataframe

         Date Revenue
0  2023-12-31   25167
1  2023-09-30   23350
2  2023-06-30   24927
3  2023-03-31   23329
4  2022-12-31   24318


<strong><span style="color: #FF33F0">4. Process the DataFrame: clean up the rows to get clean values by removing $ and commas. Remove also those that are empty or have no information.</span></strong>

In [7]:
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace('$', '').str.replace(',', '')
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != '']
tesla_revenue.head()

/tmp/ipykernel_1211/983950017.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace('$', '').str.replace(',', '')


,Date,Revenue
0,2023-12-31,25167
1,2023-09-30,23350
2,2023-06-30,24927
3,2023-03-31,23329
4,2022-12-31,24318


<strong><span style="color: #FF33F0">5. Store the data in sqlite: Create an empty instance of the database and include the clean data in it, as we saw in the database module. Once you have an empty database</span></strong>

In [11]:
import sqlite3

# create a connection to the database
conn = sqlite3.connect('Tesla.db')
print(conn)
print('Opened database successfully')

Opened database successfully


In [18]:
# create a cursor object using the cursor() method
cursor = conn.cursor()

# create a table in the database
cursor.execute('''CREATE TABLE revenue
               (Date TEXT PRIMARY KEY NOT NULL,
               Revenue TEXT NOT NULL);''')

print('Table created successfully')

# cursor.execute('DROP TABLE IF EXISTS revenue')

Table created successfully


In [20]:
# insert the values
tesla_tuples = list(tesla_revenue.to_records(index=False)) 
print(tesla_tuples[:5])

cursor.executemany('INSERT INTO revenue (Date, Revenue) VALUES (?, ?)', tesla_tuples)

[('2023-12-31', '25167'), ('2023-09-30', '23350'), ('2023-06-30', '24927'), ('2023-03-31', '23329'), ('2022-12-31', '24318')]


In [21]:
# commit the changes
conn.commit()

In [22]:
# check if the data has been inserted into the database
cursor.execute('SELECT * FROM revenue')
rows = cursor.fetchall()
for row in rows:
    print(row)

('2023-12-31', '25167')
('2023-09-30', '23350')
('2023-06-30', '24927')
('2023-03-31', '23329')
('2022-12-31', '24318')
('2022-09-30', '21454')
('2022-06-30', '16934')
('2022-03-31', '18756')
('2021-12-31', '17719')
('2021-09-30', '13757')
('2021-06-30', '11958')
('2021-03-31', '10389')
('2020-12-31', '10744')
('2020-09-30', '8771')
('2020-06-30', '6036')
('2020-03-31', '5985')
('2019-12-31', '7384')
('2019-09-30', '6303')
('2019-06-30', '6350')
('2019-03-31', '4541')
('2018-12-31', '7226')
('2018-09-30', '6824')
('2018-06-30', '4002')
('2018-03-31', '3409')
('2017-12-31', '3288')
('2017-09-30', '2985')
('2017-06-30', '2790')
('2017-03-31', '2696')
('2016-12-31', '2285')
('2016-09-30', '2298')
('2016-06-30', '1270')
('2016-03-31', '1147')
('2015-12-31', '1214')
('2015-09-30', '937')
('2015-06-30', '955')
('2015-03-31', '940')
('2014-12-31', '957')
('2014-09-30', '852')
('2014-06-30', '769')
('2014-03-31', '621')
('2013-12-31', '615')
('2013-09-30', '431')
('2013-06-30', '405')
('2013-0

<strong><span style="color: #FF33F0">6: Visualize the data: Suggest at least 3 and plot them.</span></strong>